# Ekerå–Håstad's algorithm for factoring RSA integers
This notebook exemplifies using [Quaspy](https://github.com/ekera/quaspy) to simulate Ekerå–Håstad's algorithm [[EH17]](https://doi.org/10.1007/978-3-319-59879-6_20) for factoring RSA integers, with improvements from [[E20]](https://doi.org/10.1007/s10623-020-00783-2) and [[E23p]](https://doi.org/10.48550/arXiv.2309.01754), and with the post-processing from [[E20]](https://doi.org/10.1007/s10623-020-00783-2) and [[E23p]](https://doi.org/10.48550/arXiv.2309.01754) when making and not making tradeoffs, respectively.

To start off, let us pick two distinct prime factors $p$ and $q$ uniformly at random from the set of all $l$-bit prime factors, under the restriction that $N = pq$ is a $2l$-bit integer, so as to simulate how the modulus $N$ is typically selected in RSA. To this end, we use the [<code>sample_l_bit_prime(l)</code>](../docs/math/primes/sample_l_bit_prime.md) convenience function provided by [Quaspy](https://github.com/ekera/quaspy).

In [1]:
!pip3 install -q quaspy # Make sure that quaspy is installed.

In [2]:
from quaspy.math.primes import sample_l_bit_prime;

l = 1024;

while True:
  p = sample_l_bit_prime(l);

  while True:
    q = sample_l_bit_prime(l);
    if p != q:
      break;

  N = p * q;

  if (2 ** (2 * l - 1)) <= N < (2 ** (2 * l)):
    break;

print("Sampled p =", p);
print("Sampled q =", q);

print("\nComputed N =", N);

Sampled p = 114048533234126441812980005049651233053294867169283857236888887278168563271677898984999378320672379491025683444245614088265428688230539658601640616678044793052262419517400705367869608057399277237146015950013819489793222781535813238701423397297991775186995938847486109444242061076407024032808367935436200018369
Sampled q = 167980674112484011291966602366746447748322627380129498005723593836764995154714026157245918928756225435659944168039901282589194040925039094377602521640249254823306340950954690246224272188389189864597408169168275011929302286122789957707201740156084975927191050810099737891662809121304033694308720801433580524913

Computed N = 191579494942085959971058702063247825130174558433308416687972811446089498732666985939547221689513987632405936030927857308653956654479531584977257360363758631307037315300163953939016823424431371118325238020690616074270171855093672938257158708435106621380221386566106571652380403145842537126018228639769210419788788032798052992342794970519613494

## 1. Simulating order finding in $\mathbb Z_N^*$ exactly to sample $r$
The first step in the classical pre-processing in Ekerå–Håstad's factoring algorithm [[EH17]](https://doi.org/10.1007/978-3-319-59879-6_20) is to select $g$ uniformly at random from $\mathbb Z_N^*$.

[Quaspy](https://github.com/ekera/quaspy) provides a function [<code>sample_r_given_N(N, factors)</code>](../docs/factoring/sampling/sample_r_given_N.md) for exactly sampling an element $g$ uniformly at random from $\mathbb Z_N^*$ and returning its order without explicitly computing and returning $g$.
This function requires the factorization of $N$ to be known, as is the case here given that we select the factors of $N$.
For further details, see [[E24t]](https://kth.diva-portal.org/smash/get/diva2:1902626/FULLTEXT01.pdf) (see Sect. 5.2.3).

Below, we use the [<code>sample_r_given_N(N, factors)</code>](../docs/factoring/sampling/sample_r_given_N.md) function to simulate the initial classical pre-processing step of Ekerå–Håstad's algorithm.
We then setup a simulated generator $g$ for a cyclic group of order $r$ by using the [SimulatedCyclicGroupElement](../docs/math/groups/SimulatedCyclicGroupElement.md) class provided by [Quaspy](https://github.com/ekera/quaspy).

In [3]:
from quaspy.factoring.sampling import sample_r_given_N;

from quaspy.math.groups import SimulatedCyclicGroupElement;

# Sample r.
r = sample_r_given_N(N, factors = [[p, 1], [q, 1]]);

print("Sampled r =", r);

# Setup a simulated cyclic group element of order r.
g = SimulatedCyclicGroupElement(r);

Sampled r = 21381640060500665175341372998130337626135553396574600076782679848893917269270868966467323849276114691116733932023198360340843376616019150109068901826312347244088986082607584144979556185762429812313084600523506258289081680255990283287629320137846721136185422607824394157631741422527068875671677303545670805779683899634440500871846596478286776570418268228225999576795440760953777930079938398918549274789636247679315524549032976559926519796197844702971556907104726233481839436930733815345117989297019230977868487009432164006997906854141553948014679698515794566462320187278119937607890820395895228137545761627925760696


Note that an advantage of using the above method to sample $r$ is that the method is exact. A disadvantage is that it does not explicitly compute and return $g$, making it necessary to simulate $g$. In turn, this may lead to the classical pre- and post-processing appearing to be faster than it actually is in practice in what follows below, since arithmetic with simulated group elements is typically faster than arithmetic with elements of the group $\mathbb Z_N^*$.

In Sect. 2, we therefore repeat all of what follows in this section with a different heuristic sampling method that explicitly computes and returns both $g$ and $r$, where $g$ is an element of $\mathbb Z_N^*$.

### Pre-computing $x$ classically given $g$
The idea in Ekerå–Håstad's algorithm [[EH17]](https://doi.org/10.1007/978-3-319-59879-6_20), improved as in [[E20]](https://doi.org/10.1007/s10623-020-00783-2) or [[E23p]](https://doi.org/10.48550/arXiv.2309.01754), is to use $g$ to classically pre-compute

$$x = g^{(N - 1) / 2 - 2^{l - 1}} = g^{((p - 1) / 2) + ((q - 1) / 2) - 2^{l - 1}}$$

by using that $N$ is known, and to then compute the discrete logarithm $\log_g x \in [0, r) \cap \mathbb Z$ quantumly.
Provided that the order $r$ of $g$ is greater than $d = ((p - 1) / 2) + ((q - 1) / 2) - 2^{l - 1}$ — which is the case with overwhelming probability for randomly selected RSA integers — it then holds that $\log_g x = d$, in which case we can trivially solve for $p$ and $q$ given $d$ since we also know that $N = pq$ which yields a quadratic equation, the roots of which are $p$ and $q$.

In what follows below, $x$ is pre-computed given $N$. Furthermore, $d$ is computed given $p$ and $q$ since $d$ is needed by the simulator for the quantum part of Ekerå–Håstad's algorithm.

In [4]:
# Compute r.
x = g ** ((N - 1) // 2 - 2 ** (l - 1));

# Compute d.
d = ((p - 1) // 2) + ((q - 1) // 2) - 2 ** (l - 1);

# Sanity check that x = g^d.
if x != g ** d:
  raise Exception("Error: Unexpected result.");

Note that [Quaspy](https://github.com/ekera/quaspy) provides convenience functions [<code>setup_x_given_g_N(g, N)</code>](../docs/factoring/rsa/setup_x_given_g_N.md) and [<code>setup_d_given_p_q(p, q)</code>](../docs/factoring/rsa/setup_d_given_p_q.md) for computing $x$ and $d$, respectively, but above we perform the computation explicitly to facilitate reader comprehension.

### 1.1. Solving for $d$ and the factors $p$ and $q$ of $N$ in a single run
To start off, let us first consider the setting where our goal is to solve for the discrete logarithm $d$, and hence for the factors $p$ and $q$ of $N$, in a single run of the quantum part of Ekerå-Håstad's algorithm [[EH17]](https://doi.org/10.1007/978-3-319-59879-6_20).

#### 1.1.1. Sampling a frequency pair $(j, k)$
[Quaspy](https://github.com/ekera/quaspy) provides a function [<code>sample_j_k_given_d_r_tau(d, r, m, ell, tau, ..)</code>](../docs/logarithmfinding/short/sampling/sample_j_k_given_d_r_tau.md) for simulating the quantum part of Ekerå–Håstad's algorithm exactly (up to arbitrary precision) for a given logarithm $d$, order $r$ (that need not be specified), and parameters $m$, $\ell$ and $\tau$, where $m$ is an upper bound on the bit length of $d$, and where $\tau$ specifies the search interval when sampling and when solving in the classical post-processing from [[E23p]](https://doi.org/10.48550/arXiv.2309.01754).
For further details, see [[E23p]](https://doi.org/10.48550/arXiv.2309.01754).

Below, we use said function to simulate running the algorithm for $d$ and $r$ with control registers of length $m + \ell$ qubits and $\ell$ qubits respectively.
More specifically, for $g$ the generator of unknown order $r \ge 2^{m+\ell} + (2^\ell - 1)d$ and $x = g^d$, we simulate inducing the state

$$\frac{1}{2^{m + 2 \ell}}
\sum_{a, \, j \, = \, 0}^{2^{m+\ell} - 1}
\sum_{b, \, k \, = \, 0}^{2^{\ell} - 1}
\mathrm{exp}
\left(
  \frac{2 \pi \mathrm{i}}{2^{m + \ell}} (aj + 2^m bk)
\right)
|\, j, k, g^a x^{-b} \,\rangle$$

and reading out the first two control registers. This yields a frequency pair $(j, k)$ sampled from the probability distribution induced by the quantum part of Ekerå–Håstad's algorithm:

In [ ]:
from quaspy.logarithmfinding.short.sampling import sample_j_k_given_d_r_tau;

# Simulate running the quantum part of the algorithm for g and x.
m = l - 1;
Delta = 30;
ell = m - Delta;
tau = 20;

[j, k] = sample_j_k_given_d_r_tau(d, r, m, ell, tau);

print("Sampled j =", j);
print("Sampled k =", k);

Sampled j = 3390665905030106285119426660184436149295571295362562419050307938779407036045450512115753886225728885642837786397686281859767823267759791957712577879722474253511143419053343623306810395236913433177671904242969391139620253884924158990052330571261298392687756018722428060652845078570211240886377705786631632964987146042280049391190495695185973961207003142967848310196231317831511504254229802212124144137758147563942062591939212548404785721921574619551811006185023438715846740051342659727197632329847392324386206183683146575452684008851779997660377480365272882618386645292343720640375227269019301680330190423192
Sampled k = 72023202281553476421943162050704947008407149007360562261135408002792606166675071897935869277764665018032907997397109672058115272575238462650160299004326257295322533876130307813426527386853741966502038281943374127312927627512120790710361389285953669810963695290732236473080365626389787781481294132094


Above, we first use that $m = l - 1$ is an upper bound on the bit length of $d = (p - 1) / 2) + ((q - 1) / 2) - 2^{l - 1}$.
We then let $\ell = m - \Delta$ for $\Delta$ a non-negative constant.

Provided that $\Delta$ is selected sufficiently large, we then have that $r \ge 2^{m+\ell} + (2^\ell - 1)d$ as required by the analysis in [[EH17]](https://doi.org/10.1007/978-3-319-59879-6_20), [[E20]](https://doi.org/10.1007/s10623-020-00783-2) and [[E23p]](https://doi.org/10.48550/arXiv.2309.01754).
Furthermore, we then have that $\log_g x = d$ enabling us to solve $N = pq$ and $d = ((p - 1) / 2) + ((q - 1) / 2) - 2^{l - 1}$ for $p$ and $q$ by solving a quadratic equation as described above.

Meanwhile, recall that $\sim \ell$ bits of information on $d$ is computed in each run of the quantum part of Ekerå–Håstad's algorithm. Hence, provided that $\Delta$ is not selected too large, we can efficiently find $d$ in a single run.
 
Above, we select $\Delta = 30$ to strike a good compromise:
As is shown in [[E23p]](https://doi.org/10.48550/arXiv.2309.01754), the success probability is then at least $1 - 10^{-6}$ provided that we accept to perform at most $2^{28.6}$ group operations in the classical post-processing (which is clearly feasible).
For further details, see [[E23p]](https://doi.org/10.48550/arXiv.2309.01754) (in particular, see Tab. 1, and also Tab. 4).

Note also that the [<code>sample_j_k_given_d_r_tau(d, r, m, ell, tau, ..)</code>](../docs/logarithmfinding/short/sampling/sample_j_k_given_d_r_tau.md) function checks that $r \ge 2^{m+\ell} + (2^\ell - 1)d$ when $r$ is passed to the function. This is the only reason for why $r$ is passed to said function.

#### 1.1.2. Solving the frequency pair $(j, k)$ and $g$ and $x$ for $p$ and $q$
We now proceed to solve the frequency pair $(j, k)$ and $g$ and $x$ for the factors $p$ and $q$ of $N$.

To this end, we first use the [<code>solve_j_k_for_d(j, k, m, ell, g, x, tau, t, ..)</code>](../docs/logarithmfinding/short/postprocessing/solve_j_k_for_d.md) function provided by [Quaspy](https://github.com/ekera/quaspy) to solve $(j, k)$ for $d$ using the lattice-based post-processing from [[E23p]](https://doi.org/10.48550/arXiv.2309.01754). We then solve $d$ for the prime factors $p$ and $q$ of $N$ using the post-processing from [[E23p]](https://doi.org/10.48550/arXiv.2309.01754).

As stated above, to solve $d = ((p - 1) / 2) + ((q - 1) / 2) - 2^{l - 1}$ and $N = pq$ for $p$ and $q$, we only need to solve a quadratic equation.
[Quaspy](https://github.com/ekera/quaspy) provides a convenience function [<code>split_N_given_d(d, N)</code>](../docs/factoring/rsa/postprocessing/split_N_given_d.md) for this purpose, but below we solve explicitly using the quadratic formula to facilitate reader comprehension.

The parameters $\tau$ and $t$ control the search space when solving $(j, k)$ for $d$.
Below, we gradually grow the search space, from $(\tau, t) = (4, 17)$ up to $(\tau, t) = (20, 19)$ for which the success probability is at least $1 - 10^{-6}$ and for which at most $2^{28.6}$ group operations have to be performed in the classical post-processing, as stated in the previous section.
For further details, see [[E23p]](https://doi.org/10.48550/arXiv.2309.01754) (in particular, see Tab. 1).

Note that $\tau = 20$ for the last combination which explains why we specified $\tau = 20$ when sampling in the previous section.


In [6]:
from quaspy.logarithmfinding.short.postprocessing import solve_j_k_for_d;

from gmpy2 import isqrt as sqrt;

# Recover d from the pair (j, k) output by the quantum algorithm.
for [tau, t] in [[4, 17], [7, 17], [14, 17], [17, 17], [20, 19]]:
  recovered_d = solve_j_k_for_d(j, k, m, ell, g, x, tau = tau, t = t);
  if recovered_d != None:
    break;

if None == recovered_d:
  print("[FAIL] Failed to recover d.");
else:
  # Form d' = p + q given d.
  p_plus_q = 2 * recovered_d + 2 ** l + 2;

  # Solve d' = p + q and N = pq for p and q using the quadratic formula:
  candidate_p = int((p_plus_q - sqrt((p_plus_q ** 2) - 4 * N)) // 2);
  candidate_q = int((p_plus_q + sqrt((p_plus_q ** 2) - 4 * N)) // 2);

  if (1 < candidate_p < N) and \
     (1 < candidate_q < N) and \
     (candidate_p * candidate_q == N):
    print("p =", candidate_p);
    print("q =", candidate_q);

    print("\n[ OK ] Successfully recovered p and q.");
  else:
    print("[FAIL] Failed to recover p and q.");

p = 114048533234126441812980005049651233053294867169283857236888887278168563271677898984999378320672379491025683444245614088265428688230539658601640616678044793052262419517400705367869608057399277237146015950013819489793222781535813238701423397297991775186995938847486109444242061076407024032808367935436200018369
q = 167980674112484011291966602366746447748322627380129498005723593836764995154714026157245918928756225435659944168039901282589194040925039094377602521640249254823306340950954690246224272188389189864597408169168275011929302286122789957707201740156084975927191050810099737891662809121304033694308720801433580524913

[ OK ] Successfully recovered p and q.


### 1.2. Making tradeoffs and solving for the factors $p$ and $q$ of $N$ in multiple runs
Let us now consider the case where we make tradeoffs by picking $\ell \approx m/s$ for $s$ some tradeoff factor.

As explained in [[E20]](https://doi.org/10.1007/s10623-020-00783-2), each run of the quantum part of Ekerå–Håstad's algorithm yields $\sim \ell$ bits of information on the logarithm $d$.
Hence, we expect to have to perform at least $s$ runs to solve efficiently and with high probability of success in the classical post-processing.

According to the estimates in [[E20]](https://doi.org/10.1007/s10623-020-00783-2) (see e.g. Tab. 1) which were computed with the [Qunundrum](https://github.com/ekera/qunundrum) suite of MPI programs, when $m = 1023$, $s = 8$ and $\ell = \lceil m / s \rceil$, we need to make no more than $n = 9$ runs to solve efficiently in the classical post-processing with $\ge 99\%$ success probability without enumerating the lattice.
In the below example, we use this specific parameterization.

#### 1.2.1. Sampling $n$ frequency pairs $((j_1, k_1), \, \ldots, \, (j_n, k_n))$
To start off, we proceed in analogy with Sect. 1.1.1 above to sample $n$ frequency pairs $((j_1, k_1), \, \ldots, \, (j_n, k_n))$ from the distribution induced by the quantum part of Ekerå–Håstad's algorithm.
To this end, we use the [<code>sample_j_k_given_d_r(d, r, m, ell, ..)</code>](../docs/logarithmfinding/short/sampling/sample_j_k_given_d_r.md) function provided by [Quaspy](https://github.com/ekera/quaspy).

Note that the [<code>sample_j_k_given_d_r(d, r, m, ell, ..)</code>](../docs/logarithmfinding/short/sampling/sample_j_k_given_d_r.md) function is equivalent to the [<code>sample_j_k_given_d_r_tau(d, r, m, ell, tau, ..)</code>](../docs/logarithmfinding/short/sampling/sample_j_k_given_d_r_tau.md) function used above in Sect. 1.1.1, expect that it selects the search interval in a more straightforward manner that is not specifically adapted to the post-processing from [[E23p]](https://doi.org/10.48550/arXiv.2309.01754).
(And in what follows below, we will use the post-processing from [[E20]](https://doi.org/10.1007/s10623-020-00783-2), so it makes sense to call [<code>sample_j_k_given_d_r(d, r, m, ell, ..)</code>](../docs/logarithmfinding/short/sampling/sample_j_k_given_d_r.md) in this step to sample the frequency pairs.)

In [7]:
from quaspy.logarithmfinding.short.sampling import sample_j_k_given_d_r;

from math import ceil;

s = 8;
ell = ceil(m / s);
n = 9;

j_k_list = [sample_j_k_given_d_r(d, r, m, ell, timeout = 30) \
              for _ in range(n)];

print("Sampled pairs [j, k] =", j_k_list);

Sampled pairs [j, k] = [[27055620364126457153689884333934272187776914664676209999375503375168879110858317141652515170511059435796599366116953987511433452663466964870346947540164364726249869747598061765945504536542659618267355801683170556928130376636351126605366334529963493933631015137505954740933529483068249555409627718628777319898756699281771419037743859730734223922083, 37468413344491102773796999250795435032], [18930851942850079638120904850161385747519851735413385136594308909053884648910536650175259744733680347090670959274663453881170038826396259848800589745370691477938108164568633812622920020866909857964799820310985087177280269774074007988860450248252210491149808463466227859047315916468874856132721505903470181344589002205279467193088003950015917867119, 79958613133206584029195104391492283542], [2702219736889292521450288748221941921115241083636004927541704734916639573701772306174211541892392958803727873886667233350869037984700835961489318692383673913043847266931927876630102833581015585

#### 1.2.2. Solving the $n$ frequency pairs $((j_1, k_1), \, \ldots, \, (j_n, k_n))$ and $g$ and $x$ for $p$ and $q$
We now proceed to solve the $n$ frequency pairs $((j_1, k_1), \, \ldots, \, (j_n, k_n))$ and $g$ and $x$ for the factors $p$ and $q$ of $N$.

To this end, we first use the [<code>solve_multiple_j_k_for_d(j_k_list, m, l, g, x, ..)</code>](../docs/logarithmfinding/short/postprocessing/solve_multiple_j_k_for_d.md) function provided by [Quaspy](https://github.com/ekera/quaspy).
It jointly solves $((j_1, k_1), \, \ldots, \, (j_n, k_n))$ for $d$ by using the lattice-based post-processing from [[E20]](https://doi.org/10.1007/s10623-020-00783-2).
We then solve $d$ and $N$ for the prime factors $p$ and $q$ of $N$ by using the [<code>split_N_given_d(d, N)</code>](../docs/factoring/rsa/postprocessing/split_N_given_d.md) convenience function provided by [Quaspy](https://github.com/ekera/quaspy).

In [8]:
from quaspy.logarithmfinding.short.postprocessing import solve_multiple_j_k_for_d;

from quaspy.factoring.rsa.postprocessing import split_N_given_d;

recovered_d = solve_multiple_j_k_for_d(j_k_list,
                                       m,
                                       ell,
                                       g,
                                       x,
                                       enumerate = False,
                                       timeout = 30);

if None == recovered_d:
  print("[FAIL] Failed to recover d).");
else:
  result = split_N_given_d(d, N);

  if None != result:
    [recovered_p, recovered_q] = result;

    print("p =", recovered_p);
    print("q =", recovered_q);

    print("\n[ OK ] Successfully recovered p and q.");
  else:
    print("[FAIL] Failed to recover p and q.");

p = 167980674112484011291966602366746447748322627380129498005723593836764995154714026157245918928756225435659944168039901282589194040925039094377602521640249254823306340950954690246224272188389189864597408169168275011929302286122789957707201740156084975927191050810099737891662809121304033694308720801433580524913
q = 114048533234126441812980005049651233053294867169283857236888887278168563271677898984999378320672379491025683444245614088265428688230539658601640616678044793052262419517400705367869608057399277237146015950013819489793222781535813238701423397297991775186995938847486109444242061076407024032808367935436200018369

[ OK ] Successfully recovered p and q.


## 2. Simulating order finding in $\mathbb Z_N^*$ heuristically to sample $g$ and $r$
In addition to the [<code>sample_r_given_N(N, factors)</code>](../docs/factoring/sampling/sample_r_given_N.md) function used in [Sect. 1](#) above, [Quaspy](https://github.com/ekera/quaspy) also provides a function [<code>sample_g_r_given_N(N, N_factors, ..)</code>](../docs/factoring/sampling/sample_g_r_given_N.md) for sampling an element $g$ uniformly at random from $\mathbb Z_N^*$ and heuristically computing and returning its order $r$ alongside $g$.
For further details, see [[E21b]](https://doi.org/10.1007/s11128-021-03069-1) (see App. A).

Note that the [<code>sample_g_r_given_N(N, N_factors, ..)</code>](../docs/factoring/sampling/sample_g_r_given_N.md) function requires the factorization of $N = pq$ to be known, as is the case here given that we selected $p$ and $q$. (If the function is also fed the factorization of $p - 1$ and $q - 1$ then the function can be made exact, as explained in [[E21b]](https://doi.org/10.1007/s11128-021-03069-1) and in the [documentation](../docs/factoring/sampling/sample_r_given_N.md).)

Below, we use said function to simulate the initial classical pre-processing step of Ekerå–Håstad's algorithm where an element $g$ is selected uniformly at random from $\mathbb Z_N^*$. This yields $g$ and $r$:

In [9]:
from quaspy.factoring.sampling import sample_g_r_given_N;

from quaspy.math.groups import IntegerModRingMulSubgroupElement;

[g, r] = sample_g_r_given_N(N, [[p, 1], [q, 1]]);

print("Sampled g =", g);
print("Sampled r =", r);

# Setup the group element g.
g = IntegerModRingMulSubgroupElement(g, N);

Sampled g = 7875937477811175823891428746497154861986189390696914680855534708523488003366944566470768458128727697771834561968543891479889025389123382476264391679386998841635220299682945438715398873560450658337417203741080366892485005240526698554234555401426617622663575864817318146622803141457525503590986782740702428935769477571356793435883544184286851459325983357017134787139928013460688125546354446030747074429726554900790785987485011377743914078970694422455936342616638084486875312113938146629753538237083397666302147244159543356716134544004792645662715067797597323439961477172987007909551180460911258698194498422409906864631
Sampled r = 46052763207232201916119880303665342579368884238776061703839618136079206426121871620083466752287016257789888468973042622272585734249887400234917634702826594064191662331770181235340582553949848826520489908819859633238022080551363687081047766450746783985630141001467925877976058448519840655292843423021444812448549937674179540339361900107079211074747039260794460

### 2.1. Solving for the factors $p$ and $q$ of $N$ in a single run
Let us first again first consider the setting where our goal is to solve for the discrete logarithm $d$, and hence the factors $p$ and $q$, in a single run of the quantum part of Ekerå-Håstad's algorithm [[EH17]](https://doi.org/10.1007/978-3-319-59879-6_20).

#### 2.1.1. Sampling a frequency pair $(j, k)$
To start off, in analogy with Sect. 1.1.1, we use the [<code>sample_j_k_given_d_r_tau(d, r, m, ell, tau, ..)</code>](../docs/logarithmfinding/short/sampling/sample_j_k_given_d_r_tau.md) function provided by [Quaspy](https://github.com/ekera/quaspy) to simulate the quantum part of Ekerå–Håstad's factoring algorithm exactly (up to arbitrary precision) for a given logarithm $d$, order $r$, and parameters $m$, $\ell$ and $\tau$, where $m$ is an upper bound on the bit length of $d$, and where $\tau$ is related to the sampling as explained in [[E23p]](https://doi.org/10.48550/arXiv.2309.01754). This yields a frequency pair $(j, k)$ sampled from the probability distribution induced by the algorithm.

To first setup $x$ and $d$, we use the [<code>setup_x_given_g_N(g, N)</code>](../docs/factoring/rsa/setup_x_given_g_N.md) and [<code>setup_d_given_p_q(p, q)</code>](../docs/factoring/rsa/setup_d_given_p_q.md) convenience functions provided by [Quaspy](https://github.com/ekera/quaspy).

In [ ]:
from quaspy.logarithmfinding.short.sampling import sample_j_k_given_d_r_tau;

from quaspy.factoring.rsa import setup_d_given_p_q;
from quaspy.factoring.rsa import setup_x_given_g_N;

# Compute x.
x = setup_x_given_g_N(g, N);

print("Computed x =", x);

# Compute d.
d = setup_d_given_p_q(p, q);

# Sanity check that x = g^d.
if x != g ** d:
  raise Exception("Error: Unexpected result.");

m = l - 1;
Delta = 30;
ell = m - Delta;
tau = 20;

[j, k] = sample_j_k_given_d_r_tau(d, r, m, ell, tau);

print("\nSampled j =", j);
print("Sampled k =", k);

Computed x = 12603574073955776020385599073173485249876757028444260800780124566966487921565105689557961945551511121901149050397946320245826340572300966194868129305245948174353619866577297471118021088335955427429966814214156477913821628207185653056127118321255523121199902012551820268347761448566349920309895399584524778453217101545035002337121537677046117013859627271772224044049151774422978921194448809732006351019534254403475978763701391707735700995079573425778713087630746077133501329332349061475111766284683028040747794788536543459150777166969565813420299486488880386548205524286702891219383078723853576219606314266563495864134 (mod 1915794949420859599710587020632478251301745584333084166879728114460894987326669859395472216895139876324059360309278573086539566544795315849772573603637586313070373153001639539390168234244313711183252380206906160742701718550936729382571587084351066213802213865661065716523804031458425371260182286397692104197887880327980529923427949705196134948789638582703990897605

#### 2.1.2. Solving the frequency pair $(j, k)$ and $g$ and $x$ for $p$ and $q$
We may then proceed to solve the frequency pair $(j, k)$ and $g$ and $x$ for the factors $p$ and $q$ of $N$.

To this end, in analogy with Sect. 1.1.2, we first use the [<code>solve_j_k_for_d(j, k, m, ell, g, x, tau, t ..)</code>](../docs/logarithmfinding/short/postprocessing/solve_j_k_for_d.md) function provided by [Quaspy](https://github.com/ekera/quaspy) to solve $(j, k)$ for $d$ using the lattice-based post-processing from [[E23p]](https://doi.org/10.48550/arXiv.2309.01754).
We then solve $d$ and $N$ for the prime factors $p$ and $q$ of $N$ by using the [<code>split_N_given_d(d, N)</code>](../docs/factoring/rsa/postprocessing/split_N_given_d.md) convenience function provided by [Quaspy](https://github.com/ekera/quaspy).

In [11]:
from quaspy.logarithmfinding.short.postprocessing import solve_j_k_for_d;

from quaspy.factoring.rsa.postprocessing import split_N_given_d;

# Recover d.
for [tau, t] in [[4, 17], [7, 17], [14, 17], [17, 17], [20, 19]]:
  recovered_d = solve_j_k_for_d(j, k, m, ell, g, x, tau = tau, t = t);
  if recovered_d != None:
    break;

if None == recovered_d:
  print("[FAIL] Failed to recover d.");
else:
  result = split_N_given_d(d, N);

  if None != result:
    [recovered_p, recovered_q] = result;

    print("p =", recovered_p);
    print("q =", recovered_q);

    print("\n[ OK ] Successfully recovered p and q.");
  else:
    print("[FAIL] Failed to recover p and q.");

p = 167980674112484011291966602366746447748322627380129498005723593836764995154714026157245918928756225435659944168039901282589194040925039094377602521640249254823306340950954690246224272188389189864597408169168275011929302286122789957707201740156084975927191050810099737891662809121304033694308720801433580524913
q = 114048533234126441812980005049651233053294867169283857236888887278168563271677898984999378320672379491025683444245614088265428688230539658601640616678044793052262419517400705367869608057399277237146015950013819489793222781535813238701423397297991775186995938847486109444242061076407024032808367935436200018369

[ OK ] Successfully recovered p and q.


### 2.2. Making tradeoffs and solving for the factors $p$ and $q$ of $N$ in multiple runs
Let us now consider the case where we make tradeoffs by picking $\ell \approx m/s$ for $s$ some tradeoff factor.
As explained in [[E20]](https://doi.org/10.1007/s10623-020-00783-2), each run of the quantum algorithm then yields $\sim \ell$ bits of information on the logarithm $d$.

#### 2.2.1. Sampling $n$ frequency pairs $((j_1, k_1), \, \ldots, \, (j_n, k_n))$
To start off, in analogy with Sect. 1.2.1, we proceed to sample $n$ frequency pairs $((j_1, k_1), \, \ldots, \, (j_n, k_n))$ from the distribution induced by the quantum part of Ekerå–Håstad's algorithm.
To this end, we use the [<code>sample_j_k_given_d_r(d, r, m, ell, ..)</code>](../docs/logarithmfinding/short/sampling/sample_j_k_given_d_r.md) function provided by [Quaspy](https://github.com/ekera/quaspy).

In [12]:
from quaspy.logarithmfinding.short.sampling import sample_j_k_given_d_r;

from math import ceil;

s = 8;
ell = ceil(m / s);
n = 9;

j_k_list = [sample_j_k_given_d_r(d, r, m, ell, timeout = 30) \
              for _ in range(n)];

print("Sampled pairs [j, k] =", j_k_list);

Sampled pairs [j, k] = [[29924180134257712155876132858563433751258519473771827780101748500151675908447499905065978717295398512823372515365657774391781724997597846009462580563469670728728698769888569725842732280316127743218064781445939079355522274650826015738387070938575271304421217076022697867942957857846269423196820325922325048678135604425507584362169803088872039062178, 111638671264799981438912610674328028673], [10522637235991480208880359420388869533590572022002639700088801432701505644841584578955056806549150818928442066590903647028090334259136166410773522608336491878036973880521055120739654317447351761647612383453890304818932466900833147284160630254294948095558778235469324081508290967813662504357853583314838232581404406910376530936726319567281105043415, 283595004726212002765315576397515511417], [22879551371711310397031118909250871550491944838635068354904296242478403208657304978944106511581575808568508725770008324457249785009328211897979740020706655705132480813923683437336977446750833

#### 2.2.2. Solving the $n$ frequency pairs $((j_1, k_1), \, \ldots, \, (j_n, k_n))$ and $g$ and $x$ for $p$ and $q$
We now proceed to solve the $n$ frequency pairs $((j_1, k_1), \, \ldots, \, (j_n, k_n))$ and $g$ and $x$ for the factors $p$ and $q$ of $N$.

To this end, in analogy with Sect. 1.2.2, we first use the [<code>solve_multiple_j_k_for_d(j_k_list, m, l, g, x, ..)</code>](../docs/logarithmfinding/short/postprocessing/solve_multiple_j_k_for_d.md) function provided by [Quaspy](https://github.com/ekera/quaspy).
It jointly solves $((j_1, k_1), \, \ldots, \, (j_n, k_n))$ for $d$ by using the lattice-based post-processing from [[E20]](https://doi.org/10.1007/s10623-020-00783-2).
We then solve $d$ and $N$ for the prime factors $p$ and $q$ of $N$ by using the [<code>split_N_given_d(d, N)</code>](../docs/factoring/rsa/postprocessing/split_N_given_d.md) convenience function provided by [Quaspy](https://github.com/ekera/quaspy).

In [13]:
from quaspy.logarithmfinding.short.postprocessing import solve_multiple_j_k_for_d;

from quaspy.factoring.rsa.postprocessing import split_N_given_d;

recovered_d = solve_multiple_j_k_for_d(j_k_list,
                                       m,
                                       ell,
                                       g,
                                       x,
                                       enumerate = False,
                                       timeout = 30);

if None == recovered_d:
  print("[FAIL] Failed to recover d.");
else:
  result = split_N_given_d(d, N);

  if None != result:
    [recovered_p, recovered_q] = result;

    print("p =", recovered_p);
    print("q =", recovered_q);

    print("\n[ OK ] Successfully recovered p and q.");
  else:
    print("[FAIL] Failed to recover p and q.");

p = 167980674112484011291966602366746447748322627380129498005723593836764995154714026157245918928756225435659944168039901282589194040925039094377602521640249254823306340950954690246224272188389189864597408169168275011929302286122789957707201740156084975927191050810099737891662809121304033694308720801433580524913
q = 114048533234126441812980005049651233053294867169283857236888887278168563271677898984999378320672379491025683444245614088265428688230539658601640616678044793052262419517400705367869608057399277237146015950013819489793222781535813238701423397297991775186995938847486109444242061076407024032808367935436200018369

[ OK ] Successfully recovered p and q.
